# DCGAN in Theory and Practice

<a href="https://colab.research.google.com/github/zurutech/gans-from-theory-to-production/blob/master/3.%20TFGAN/3.1.%20DCGAN%20in%20Theory%20and%20Practice.ipynb">
<img align="left" src="https://cdn-images-1.medium.com/max/800/1*ZpNn76K98snC9vDiIJ6Ldw.jpeg"></img>
</a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Theory" data-toc-modified-id="Theory-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Theory</a></span><ul class="toc-item"><li><span><a href="#Enter-DCGAN" data-toc-modified-id="Enter-DCGAN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Enter DCGAN</a></span></li><li><span><a href="#Generator:-from-noise-to-insight" data-toc-modified-id="Generator:-from-noise-to-insight-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generator: from noise to insight</a></span><ul class="toc-item"><li><span><a href="#Deconvolution" data-toc-modified-id="Deconvolution-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Deconvolution</a></span></li><li><span><a href="#Batch-Normalization" data-toc-modified-id="Batch-Normalization-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Batch Normalization</a></span></li><li><span><a href="#generator_fn()" data-toc-modified-id="generator_fn()-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>generator_fn()</a></span></li></ul></li><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Discriminator</a></span><ul class="toc-item"><li><span><a href="#discriminator_fn()" data-toc-modified-id="discriminator_fn()-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>discriminator_fn()</a></span></li></ul></li><li><span><a href="#Loss-function:-a-bridge-between-two-networks" data-toc-modified-id="Loss-function:-a-bridge-between-two-networks-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Loss function: a bridge between two networks</a></span></li></ul></li><li><span><a href="#Practice---Introduction" data-toc-modified-id="Practice---Introduction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Practice - Introduction</a></span><ul class="toc-item"><li><span><a href="#Input-Functions" data-toc-modified-id="Input-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Input Functions</a></span><ul class="toc-item"><li><span><a href="#Loading-Images" data-toc-modified-id="Loading-Images-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Loading Images</a></span></li></ul></li><li><span><a href="#Hyperparameters" data-toc-modified-id="Hyperparameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Hyperparameters</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#TensorBoard" data-toc-modified-id="TensorBoard-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>TensorBoard</a></span></li></ul></li><li><span><a href="#Predictions" data-toc-modified-id="Predictions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Predictions</a></span><ul class="toc-item"><li><span><a href="#predict_input_fn()" data-toc-modified-id="predict_input_fn()-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>predict_input_fn()</a></span></li></ul></li><li><span><a href="#Preparing-for-Production" data-toc-modified-id="Preparing-for-Production-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preparing for Production</a></span><ul class="toc-item"><li><span><a href="#serving_input_receiver_fn" data-toc-modified-id="serving_input_receiver_fn-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>serving_input_receiver_fn</a></span></li><li><span><a href="#Exporting-the-model-for-production" data-toc-modified-id="Exporting-the-model-for-production-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Exporting the model for production</a></span></li><li><span><a href="#Local-CloudML-Predictions-testing" data-toc-modified-id="Local-CloudML-Predictions-testing-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Local CloudML Predictions testing</a></span><ul class="toc-item"><li><span><a href="#Generate-the-test-noise" data-toc-modified-id="Generate-the-test-noise-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Generate the test noise</a></span></li><li><span><a href="#Execute-the-test" data-toc-modified-id="Execute-the-test-4.3.2"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Execute the test</a></span></li></ul></li></ul></li><li><span><a href="#To-the-serving" data-toc-modified-id="To-the-serving-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>To the serving</a></span></li><li><span><a href="#NOTES" data-toc-modified-id="NOTES-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>NOTES</a></span></li><li><span><a href="#Links" data-toc-modified-id="Links-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Links</a></span></li></ul></div>

In [1]:
import altair as alt
import numpy as np
import tensorflow as tf

tfgan = tf.contrib.gan

## Theory

### Enter DCGAN

![DCGAN](images/dcgan.png)

If you take a look at an [impressive list of GANs](), you see that DCGAN, the architecture of our choice, is just a small drop in the ocean. While there may be sexier ones, very few offers the same package of clarity, stability, performances and computational efficiency, these are the main one reason why DCGAN is considered one of the cornerstones of this field.

Proposed in a 2015 paper by Alec Radford, Luke Metz, Soumith Chintala, the idea behind DCGAN is quite straightforward: combining a set of architectural constraints with the power of CNN yields us a robust, stable, and competitive model. Succeeding where many others failed, the authors present us with a simple model, asymmetrical architecture, 4 deconvolutional layers at the Generator with 4 layers of convolution at the Discriminator with the following constraints:

- All pooling layers are replaced with stranded convolutions (discriminator) and fractional-stride convolutions (generator).
- Batch-normalization used in both networks.
- Removal of fully connected layers (except for the discriminator output)
- `ReLU` for all Generator layers except the output, which uses `tanh`.
- `LeakyReLU` activation in the discriminator for all layers.

Although there has been very recent advancements in state of the art (i.e., CoordConv, Spectral Normalization), we felt that before venturing into the bleediest of the edges it is essential to have a firm understanding of the basic concept. We leave you (an opinionated) list of further resources you can use to get up to speed to the most exciting line of research.

### Generator: from noise to insight

![DCGAN Generator](images/dcgan_generator.png)

If you recall the theoretical explanation, the Generator is the network responsible for the data-generation, learning how to fool its discriminator allows it to produce realistic results "sampled' from the learned manifold.

The most common type of generator_inputs is pure and straightforward noise; however, more specialized GANs may require additional parameters. Since our full-demo uses a Deep Convolutional GAN (DCGAN), we don't need any other parameters.

#### Deconvolution

> When we have neural networks generate images, we often have them build them up from low resolution, high-level 
> descriptions. This allows the network to describe the rough image and then fill in the details.
>
> In order to do this, we need some way to go from a lower resolution image to a higher one. We generally do this with the deconvolution operation. 
> Roughly, deconvolution layers allow the model to use every point in the small image to “paint” a square in the larger one.

We have implemented the deconvolution layers following the findings explained [here]().

The core idea behind the paper is that boundaries and checkerboards artifacts are not due to the adversarial training but are inherent to the deconvolution process, to tackle this issue, the suggested solution is an improved version of the upsampling operation, called the **different resize-convolution** whose core is a resize operation powered by either **nearest-neighbor interpolation** or **bilinear interpolation**. 

As in the paper, we are going with the ** nearest-neighbor** one.

In [2]:
def deconv2d(inputs, filters, strides=(1, 1), activation=tf.nn.relu):
    """\"Deconvolution\" layer.
    
    It uses upsampling with nearest neighbor interpolation to reduce the
    presence of checkboard artifacts.
    """
    
    input_h, input_w = inputs.shape[1].value, inputs.shape[2].value
    layer_1 = tf.image.resize_nearest_neighbor(
        inputs, (2 * input_h, 2 * input_w), name="NNUpSample2D"
    )
    # Padding before convolution is used to reduce boundary artifacts
    layer_1 = tf.pad(layer_1, [[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT")
    layer_2 = tf.layers.conv2d(
        inputs=layer_1,
        filters=filters,
        kernel_size=5,
        padding="valid",
        use_bias=False,
        activation=activation,
        strides=strides,
    )
    return layer_2

#### Batch Normalization

We do not enter into the theoretical depth of `BatchNorm` as the discussion on why or whether it works it's still open and going [NOTE](). What you need to know is that TensorFlow makes it very easy to implement such an operation.

```python
tf.layers.batch_normalization(input, training=is_training)
```


#### generator_fn()

In [3]:
def generator_fn(inputs, mode):
    """Generator producing images from noise.
        Args:
            noise: A single Tensor representing noise.
            mode: Either TRAIN, EVAL or PREDICT
        Returns:
            A 64x64 (None, 4096) flattened tensor whose values are
            inside the (-1, 1) range."""
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    linear = tf.layers.dense(inputs=inputs, units=1024 * 4 * 4, activation=tf.nn.relu)
    net = tf.reshape(linear, (-1, 4, 4, 1024))
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 512)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 256)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 128)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 64)
    net = tf.layers.conv2d(
        inputs=net,
        filters=3,
        kernel_size=5,
        padding="same",
        data_format="channels_last",
        use_bias=False,
        strides=(1, 1),)
    output = tf.tanh(net)
    return output

### Discriminator

![DCGAN Discriminator](images/dcgan_discriminator.png)

As for the `input_fn()` we can reuse all the code we defined for the vanilla `Estimator` describing the discriminator network architecture.

#### discriminator_fn()

In [4]:
def custom_conv2d(inputs, filters, strides=(2,2)):
    """Helper layer used to instatiate `tf.layers.conv2d` with proper arguments."""
    layer_1 = tf.layers.conv2d(
        inputs=inputs,
        filters=filters,
        kernel_size=5,
        padding="same",
        data_format="channels_last",
        use_bias=False,
        strides=strides,
    )
    layer_1 = tf.nn.leaky_relu(layer_1, alpha=0.2)

    return layer_1


In [5]:
def discriminator_fn(inputs, conditioning, mode):
    """Build the Discriminator network.
    Args:
        features: a batch of images to classify, expected input shape (None, 64, 64 , 3)
        labels: a batch of labels
        mode: the tf.estimator.ModeKey
        params: a dict of optional parameters
    Returns:
            The output (logits) of the discriminator.
    """
    
    # In every mode, define the model
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    net = custom_conv2d(inputs, filters=64)
    net = custom_conv2d(inputs, filters=128)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = custom_conv2d(net, filters=256)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = custom_conv2d(net, filters=512)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = custom_conv2d(net, filters=1024)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = tf.reshape(net, (-1, net.shape[1] * net.shape[2] * net.shape[3]))
    output = tf.layers.dense(net, units=1)

    return output

### Loss function: a bridge between two networks

There are a lot of loss functions usable in GANs'architectures from very domain-specific ones to ones that are perfect for general use cases. Since our goal here falls in the latter category, we use the so-called **Non-Saturating Loss** which is the non-saturating variant of the **MinMax Loss** proposed by Goodfellow in the [original paper]().

As stated earlier, one of the beauties of `TFGAN` is its offerings losses ready for use, if you cannot find the loss you want, you can create your own.

We use the following two losses:

```python
generator_loss_fn=tfgan.losses.minimax_generator_loss

discriminator_loss_fn=tfgan.losses.minimax_discriminator_loss
```

It is that easy.

## Practice - Introduction

Having seen the general structure of the TFGAN library, we dive right into the model architecture.

In our showcasing of the TensorFlow API, we have built an image recognition network, GANs, however, require the training of both a Generator and a Discriminator together. While this task is either verbose and possibly performance-constrained (in the case of writing it in vanilla TensorFlow) or complicated (Estimator API) thanks to GANEstimator it becomes remarkably simple.

### Input Functions

The input function for the `GANEstimator` is very much the same as the one we would define for a normal `tf.Estimator`.


>    The function should construct and return one of the following:
>
>    A tf.data.Dataset object: Outputs of Dataset object must be a tuple (features, labels) with same constraints as below.
>
>    A tuple (features, labels): Where `features` is a Tensor or a dictionary of string feature name to Tensor and `labels` is a Tensor or a dictionary of string label name to Tensor. Model_fn consumes both features and labels. They should satisfy the expectation of model_fn from inputs.


We write our input function following the style and structure presented available on [TensorFlow Models](#1) as an example.

> ```python
def _get_train_input_fn(batch_size, noise_dims, dataset_dir=None,
                        num_threads=4):
  def train_input_fn():
    with tf.device('/cpu:0'):
      images, _, _ = data_provider.provide_data(
          'train', batch_size, dataset_dir, num_threads=num_threads)
    noise = tf.random_normal([batch_size, noise_dims])
    return noise, images
return train_input_fn
>```

The two main ideas to take from this are: 

1. structure the code in the same way, the `GANEstimator` wants **function objects** as inputs thus the need to wrap them. 
2. Return **noise** first and then the **real_data** (here called `images`) and optionally **labels**.

[NOTE]()

#### Loading Images

We reuse the same `input_fn()` we defined earlier with minor changes in order to make it consistent with the aforementioned code strfucture.

In [6]:
def _get_train_images_input_fn(file_pattern, image_size=(64, 64, 3), shuffle=False,
                 batch_size=32, num_epochs=None, buffer_size=4096):
    """get_input_fn exploits the `file_pattern` to create an input_fn that reads all the content
    of the specified pattern, creating an object dataset.
    
    Args:
        file_pattern: python string, the pattern of the file to read to generate the dataset
        image_size: the new size of the read images
        shuffle: True if the order of the elements in the generated dataset shold be randomized
        batch_size: the size of the batches
        num_epochs: the number of epochs to repeat the dataset before throwing an exeption; None is unlimited
        buffer_size: how many images read before starting to generate output
    Returns:
        input_fn: the generated input_fn that returns a correctly instantiated iterator
    """
    
    def _img_string_to_tensor(image_string):
        """Decode an image as read from a `tf.decode_raw`, scales it between 0-1 and resize the
        image as specified in the parent method.
        Args:
            image_string: the raw image tensor
        Returns:
            image_resize: image in [0,1] correctly resized
        """
        
        nonlocal image_size
        
        image_decoded = tf.image.decode_jpeg(image_string, channels=image_size[-1])
        # The conversion to float automatically scales the values in [0., 1.]
        image_decoded_as_float = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)
        image_decoded = (image_decoded_as_float - 0.5) * 2
        image_resized = tf.image.resize_images(image_decoded, size=image_size[:2])
        

        return image_resized

    def _path_to_img(path):
        """Given the path of an image, returns the pair (image, label)
        where image is the corretly resized image, and label is the label associated with it.
        Args:
            path: the path of the image to read
        Returns:
            (image_resized, label): the image, label pair associated the path
        """
        
        # Get the parent folder of this file to get its class
        # Associate the label 0 to dogs and 1 to cats
        label = tf.cond(
                    tf.equal(tf.string_split([path], delimiter='/').values[-2], "dogs"),
                    lambda: 0, lambda: 1)

        image_string = tf.read_file(path) # read image and process it
        image_resized = _img_string_to_tensor(image_string)

        return image_resized, label
    
    def _input_fn():
        """The input function that builds the `tf.data.Dataset` object and instantiate
        the iterator correctly ready to be use.
        Returns:
            the iterator associated to the built Dataset object.
        """
        
        # Use the static method `list_files` that builds a dataset of all
        # files matching this pattern.
        dataset_path = tf.data.Dataset.list_files(file_pattern)

        if shuffle:
            dataset_path = dataset_path.apply(tf.contrib.data.shuffle_and_repeat(buffer_size, num_epochs))
        else:
            dataset_path = dataset_path.repeat(num_epochs)

        # The map function maps the path to the pair (image, label)
        dataset = dataset_path.map(_path_to_img)
        dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
        dataset = dataset.prefetch(buffer_size)
        
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()

    return _input_fn()[0]

We define the `_get_train_input_fn()` which return a Python Callable that we instantiate and pass to `GANEstimator.train()` later on.

In [7]:
def _get_train_input_fn(file_pattern, batch_size, num_epochs, noise_dims=100, **kwargs):
    def train_input_fn():
        real_data = _get_train_images_input_fn(
                 file_pattern,
                 batch_size=batch_size, 
                 num_epochs=num_epochs)
        noise = tf.random_normal([batch_size, noise_dims], name="train_noise")
        real_data.set_shape((batch_size,) + tuple(real_data.shape[1:]))
        print(noise, real_data)
        return noise, real_data
    return train_input_fn

### Hyperparameters

Personally I like to keep my hyperparameters in a **TOML** file, loading them before the train.

Here we simply load them in a `Dict` and use `**` to unpack them.

In [8]:
def load_hyperparameters():
    hp = {
        "model_dir": "../logs/",
        "file_pattern": "../assets/celeba/*.png",
        "batch_size": 128,
        "num_epochs":1,
        "noise_dims": 100
    }
    return hp

### Training

In [9]:
def dcgan():
    
    # Set the seed at a Graph Level so that we geet consistency between runs.
    tf.set_random_seed(42)
    
    # Hyperparameters
    hyperparameters = load_hyperparameters()
    
    # Run Configuration (it has other arguments)
    run_config = tf.estimator.RunConfig(
        model_dir=hyperparameters.get("model_dir"), save_summary_steps=50, save_checkpoints_steps=500)
    
    # Instatiate the GANEstimator object
    gan_estimator = tfgan.estimator.GANEstimator(
        config=run_config,
        generator_fn=generator_fn,
        discriminator_fn=discriminator_fn,
        generator_loss_fn=tfgan.losses.modified_generator_loss,
        discriminator_loss_fn=tfgan.losses.modified_discriminator_loss,
        generator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        discriminator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        add_summaries=tfgan.estimator.SummaryType.IMAGES
    )
    
    # Instatiate the train_input_fn
    # The model will train until it exhausts the Dataset which is repeated EPOCH times
    train_input_fn = _get_train_input_fn(**hyperparameters)
    trained_model = gan_estimator.train(train_input_fn, max_steps=None)
    return trained_model

### TensorBoard

In order to track our training we need to launch a **TensorBoard** session pointing to the folder (`model_dir`) containing the logs generated by our training.

```bash
tensorboard --logdir=PATH_TO_YOUR_MODEL_DIR
```

In [10]:
trained_model = dcgan()

INFO:tensorflow:Using config: {'_model_dir': '../logs/', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6053611a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.
Tensor("train_noise:0", shape=(128, 100), dtype=float32, device=/device:CPU:0) Tensor("IteratorGetNext:0", shape=(128, 64, 64, 3), dtype=float32, device=/device:CPU:0)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Summary name Generator/dense/kernel:0 is illegal; using Generator/d

## Predictions

We train Machine Learning models because we want them to perform some specific task since we now have our GAN trained, we can finally use it to "predict" AKA generate a new image from a noise vector.

Once again, TFGAN makes it as easy as invoking the `predict()`  method of our `GANEstimator` while passing to it a `predict_input_fn` as a required argument. 

### predict_input_fn()

As previously mentioned while theoretically identical, `train_input_fn` and `predict_nput_fn()` should be implemented differently.  The first one is a simple `tf.random_normal()` node, the second should make proper use of the `tf.Dataset` API.

In [11]:
def _predict_input_fn(batch_size, noise_dims=100, **kwargs):
    
    def predict_input_fn():
        noise_gen = np.array([np.float32(np.random.normal(size=[1, noise_dims])) for i in range(batch_size)])
        dataset = tf.data.Dataset.from_tensor_slices(noise_gen)
        dataset = dataset.batch(batch_size)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()
        
    return predict_input_fn

In [12]:


predictions_batch = 3
predict_input_fn = _predict_input_fn(batch_size=predictions_batch)
predictions = trained_model.predict(predict_input_fn)
[next(predictions) for _ in range(predictions_batch)]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../logs/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[array([[[-0.6827251 , -0.7505696 , -0.8752908 ],
         [-0.82232314, -0.8630165 , -0.9298208 ],
         [-0.81369096, -0.8778263 , -0.93588006],
         ...,
         [-0.9351611 , -0.95555377, -0.96947145],
         [-0.95236045, -0.9535615 , -0.96594554],
         [-0.92229366, -0.93621176, -0.93041265]],
 
        [[-0.5353163 , -0.6326544 , -0.8305918 ],
         [-0.49425954, -0.58376014, -0.8024997 ],
         [-0.36263543, -0.5470998 , -0.7415672 ],
         ...,
         [-0.9556283 , -0.97069263, -0.9842457 ],
         [-0.9315089 , -0.94909036, -0.97653484],
         [-0.9309266 , -0.94472694, -0.967539  ]],
 
        [[-0.56928587, -0.5908825 , -0.82394737],
         [-0.60630417, -0.60422677, -0.7893984 ],
         [-0.38512808, -0.4955565 , -0.72168815],
         ...,
         [-0.91939425, -0.93203765, -0.96296644],
         [-0.92243457, -0.92333966, -0.9664244 ],
         [-0.94649196, -0.94387627, -0.974118  ]],
 
        ...,
 
        [[ 0.1841504 ,  0.1804404 

## Preparing for Production

Now that we have our model trained and ready we are just a couple steps away from being able to put our model into production.

### serving_input_receiver_fn

To serve a model in production, we first need to equip it with an interface which will receive data from our requests, such interface is specified by the aptly named `serving_input_receiver_fn`. Of the three input functions, this is the trickiest one since it has its peculiar API.

This functions requires its output to be a either a `ServingInputReceiver` or a `TensorServingInputReceiver` object; the documentation on their use is clear:

> The normal `ServingInputReceiver` always returns a feature dict, even if it contains only one entry, and so can be used only with models that accept such a dict. 
>For models that accept only a single raw feature, the `serving_input_receiver_fn` provided to `Estimator.export_savedmodel()` should return this `TensorServingInputReceiver`.

Since our model needs only a noise vector to get going, we can use `TensorServingInputReceiver`.

>The expected return values are: 
> - **features**: A single `Tensor` or `SparseTensor`, representing the feature to be passed to the model. 
> - **receiver_tensors**: A Tensor, SparseTensor, or dict of string to Tensor or SparseTensor, specifying input nodes where this receiver expects to be fed by default. Typically, this is a single placeholder expecting serialized `tf.Example` protos. 
> - **receiver_tensors_alternatives**: a dict of string to additional groups of receiver tensors, each of which may be a `Tensor`, `SparseTensor`, or dict of string to `Tensor` or `SparseTensor`. These named receiver tensor alternatives generate additional serving signatures, which may be used to feed inputs at different points within the input receiver subgraph. A typical usage is to allow feeding raw feature Tensors downstream of the `tf.parse_example()` op. Defaults to None.

In [13]:
def _serving_input_receiver_fn():
    """Instantiate a placeholder for our serving input data.

    Call the custom function `serving_input_fn()`, built following
    TensorFlow Estimator API convention, initializing the placeholder for
    the noise we will feed the model during its serving.

    The Serving Input function has two key elements:

        - the data-processing step, where we concretely prepare data to be
        fed to the:
        - Placeholder, it is the node where the input are fed.

    The things to notice is that while using `ServingInputReceiver`
    your data processing step should have at its core the parsing of the
    tf.Example received.

    With `TensorServingInputReceiver` our data won't really be passed by the
    request, instead it will have to be generated ''model-side'' inside the
    `_serving_input_receiver_fn` itself.

    Returns:
        tf.estimator.export.TensorServingInputReceiver passing the
        placeholder for the noise to it.
    """

    receiver_tensors = tf.placeholder(shape=[None, 100], dtype=tf.float32, name="serving_noise")
    return tf.estimator.export.TensorServingInputReceiver(receiver_tensors, receiver_tensors)

### Exporting the model for production

Exporting the model is as easy as calling `GANEstimator.export_savedmodel()` which the same as the normal `Estimator`.

> This method builds a new graph by first calling the serving_input_receiver_fn to obtain feature Tensors, and then calling this Estimator's model_fn to generate the model graph based on those features. It restores the given checkpoint (or, lacking that, the most recent checkpoint) into this graph in a fresh session. Finally it creates a timestamped export directory below the given export_dir_base, and writes a SavedModel into it containing a single MetaGraphDef saved from this session.

We have to specify an output folder and the `serving_input_receiver_fn`.

In [14]:
export_dir_base = "../assets/exported_models"
trained_model.export_savedmodel(
    export_dir_base, _serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from ../logs/model.ckpt-10000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../assets/exported_models/temp-b'1535124551'/saved_model.pb


b'../assets/exported_models/1535124551'

### Local CloudML Predictions testing

Before going over the required steps for model serving, we want to test it locally to make sure that the exported models will behave correctly once loaded onto CloudML Engine.

#### Generate the test noise

Google Cloud often uses Newline Delimited JSON  when working with JSON-formatted data; we can use the `jsonlines` Python library assure our compliance with the standard.

In [15]:
import jsonlines

In [16]:
BATCH_SIZE = 2
NOISE_DIMS = 100

with jsonlines.open("../assets/test_noise.ndjson", "w") as writer:
    noise = np.random.normal(size=(BATCH_SIZE, NOISE_DIMS)).tolist()
    noised_dict = [{"input": n} for n in noise]
    writer.write_all(noised_dict)

#### Execute the test

In [1]:
%%bash

MODEL_DIR="../assets/exported_models"
MODEL_ID="1535124551"
JSON_INSTANCES="../assets/test_noise.ndjson"

rm ~/google-cloud-sdk/lib/googlecloudsdk/command_lib/ml_engine/*.pyc

gcloud ml-engine local predict \
    --model-dir=$MODEL_DIR/$MODEL_ID \
    --json-instances=$JSON_INSTANCES

OUTPUT
[[[-0.21859890222549438, -0.2517372965812683, -0.25561583042144775], [-0.21977634727954865, -0.25494202971458435, -0.33070188760757446], [0.037757039070129395, -0.046547744423151016, -0.06519412994384766], [-0.0028363908641040325, -0.06904777884483337, -0.1292279213666916], [-0.046578437089920044, -0.049747053533792496, -0.12671175599098206], [-0.19817060232162476, -0.20157502591609955, -0.19760026037693024], [0.007633912842720747, -0.02761325053870678, -0.039032332599163055], [0.17688049376010895, 0.09676816314458847, 0.11660291254520416], [-0.05433105677366257, 0.07318779826164246, 0.12327593564987183], [-0.0955115258693695, -0.012160777114331722, -0.08688058704137802], [0.6173572540283203, 0.5514504909515381, 0.42657050490379333], [0.7832722067832947, 0.4643794894218445, 0.25034797191619873], [0.9063718914985657, 0.3857743740081787, 0.3882253170013428], [0.13047584891319275, -0.6152172088623047, -0.6390672326087952], [0.39360731840133667, -0.47626256942749023, -0.335339009761

2018-08-24 17:30:42.889862: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1405] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.683
pciBusID: 0000:02:00.0
totalMemory: 10.92GiB freeMemory: 10.76GiB
2018-08-24 17:30:42.889874: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1484] Adding visible gpu devices: 0
2018-08-24 17:30:43.088536: I tensorflow/core/common_runtime/gpu/gpu_device.cc:965] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-08-24 17:30:43.088556: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971]      0 
2018-08-24 17:30:43.088561: I tensorflow/core/common_runtime/gpu/gpu_device.cc:984] 0:   N 
2018-08-24 17:30:43.088737: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1097] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10409 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1)



## To the serving

Now that we have the exported model we are ready to finally serve our model.

## NOTES

[1]: [The GAN Landscape: Losses, Architectures, Regularization, and Normalization](https://arxiv.org/abs/1807.04720v1), which in our opinion is one of the most thorough scientific studies on GANs out there, suggests that `Spectral Normalization` is the real big deal and that `BatchNorm` actually hurt performance if applied to the Discriminator. We still decided to go with the classic formulation of DCGAN as not to overtax you with theoretical discussions. We leave the implementation of the `Spectral Normalization`(and SNGAN) to you as an exercise. On the theoretical treating of `BatchNorm` we recommend a back to back reading of [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167), [Understanding Batch Normalization](https://arxiv.org/abs/1806.02375v1), [How Does Batch Normalization Help Optimization? (No, It Is Not About Internal Covariate Shift)](https://arxiv.org/abs/1805.11604) 

[2]: While often you want to work with **features (and labels)** fetched from a `tf.Dataset`, the **noise** should always be instantiated using a simple TensorFlow node. Trying to create a noise-containing `tf.Dataset` for the `train_input_fn` is not worth the effort. NOTE: that as things are different for the [predict_input_fn](##predict_input_fn()). 

## Links

Generative Adversarial Networks : https://arxiv.org/abs/1406.2661

really-awesome-gan : https://github.com/nightrome/really-awesome-gan

DCGAN : https://arxiv.org/abs/1511.06434

Deconvolution and Checkerboard Artifacts : https://distill.pub/2016/deconv-checkerboard/

TFGAN MNIST GAN Example : https://github.com/tensorflow/models/tree/master/research/gan/mnist_estimator

Estimator API : https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator

export_savedmodel : https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator#export_savedmodel